In [2]:
from dotenv import load_dotenv
import os
import globus_sdk
from pathlib import Path
from prefect import flow, task, get_run_logger
import uuid
from config import Config832

# Load environment variables
load_dotenv()

# Set the client ID and fetch client secret from environment
CLIENT_ID = os.getenv('GLOBUS_CLIENT_ID')
CLIENT_SECRET = os.getenv('GLOBUS_CLIENT_SECRET')

# Define the necessary scopes for transfer
SCOPES = "urn:globus:auth:scope:transfer.api.globus.org:all"

# Initialize the ConfidentialAppAuthClient for a service account
auth_client = globus_sdk.ConfidentialAppAuthClient(CLIENT_ID, CLIENT_SECRET)
authorizer = globus_sdk.ClientCredentialsAuthorizer(auth_client, SCOPES)
transfer_client = globus_sdk.TransferClient(authorizer=authorizer)

@task(name="transfer_data_to_alcf")
def transfer_data_to_alcf(file_path: str):
    """
    Transfer data to ALCF endpoint.
    Args:
        file_path (str): Path to the file that needs to be transferred.
    """
    logger = get_run_logger()
    config = Config832()

    # Prepare the transfer data
    transfer_data = globus_sdk.TransferData(transfer_client, 
                                            config.nersc832.uuid,  
                                            config.alcfeagle_832.uuid,  
                                            label='Data Transfer to ALCF')
    transfer_data.add_item(file_path, file_path)

    # Start the transfer
    try:
        transfer_result = transfer_client.submit_transfer(transfer_data)
        logger.info(f"Transfer submitted, task ID: {transfer_result['task_id']}")
    except globus_sdk.services.transfer.errors.TransferAPIError as e:
        logger.error(f"Failed to submit transfer: {e}")
        return False

@flow(name="new_832_file_flow")
def process_new_832_file(file_path: str):
    """
    Process and transfer a file from a source to the ALCF.
    Args:
        file_path (str): Path to the file that needs to be processed.
    """
    logger = get_run_logger()
    logger.info("Starting flow for new file processing and transfer.")
    
    # Call the task to transfer data
    transfer_success = transfer_data_to_alcf(file_path)
    if not transfer_success:
        logger.error("Transfer failed due to configuration or authorization issues.")

if __name__ == "__main__":
    file = Path("/raw/transfer_tests/test.txt")
    new_file = str(file.with_name(f"test_{str(uuid.uuid4())}.txt"))
    process_new_832_file(new_file)


/opt/anaconda3/envs/globus_env/lib/python3.11/site-packages/prefect/tasks.py:356: UserWarning: A task named 'transfer_data_to_alcf' and defined at '/var/folders/7t/17b_zxx55jnggw80_6672tbh0000gn/T/ipykernel_19520/3991568951.py:24' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/opt/anaconda3/envs/globus_env/lib/python3.11/site-packages/prefect/flows.py:357: UserWarning: A flow named 'new_832_file_flow' and defined at '/var/folders/7t/17b_zxx55jnggw80_6672tbh0000gn/T/ipykernel_19520/3991568951.py:49' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


08:26:26.537 | INFO    | prefect.engine - Created flow run 'origami-tuatara' for flow 'new_832_file_flow'

08:26:26.539 | INFO    | Flow run 'origami-tuatara' - View at https://app.prefect.cloud/account/ca79a138-2e4f-47c8-8d00-4908b002539f/workspace/c1a9883b-6687-4812-b00e-e520ad98a996/flow-runs/flow-run/554a481a-9c5e-4a37-8d37-c2b5e019bb46

08:26:26.785 | INFO    | Flow run 'origami-tuatara' - Starting flow for new file processing and transfer.

08:26:26.958 | INFO    | Flow run 'origami-tuatara' - Created task run 'transfer_data_to_alcf-0' for task 'transfer_data_to_alcf'

08:26:26.961 | INFO    | Flow run 'origami-tuatara' - Executing 'transfer_data_to_alcf-0' immediately...

08:26:28.614 | ERROR   | Task run 'transfer_data_to_alcf-0' - Failed to submit transfer: ('POST', 'https://transfer.api.globus.org/v0.10/transfer', 'Bearer', 403, 'ConsentRequired', 'Missing required data_access consent', 'SxxiEhy6d')

08:26:28.770 | INFO    | Task run 'transfer_data_to_alcf-0' - Finished in state Completed()

08:26:28.783 | ERROR   | Flow run 'origami-tuatara' - Transfer failed due to configuration or authorization issues.

08:26:29.032 | INFO    | Flow run 'origami-tuatara' - Finished in state Completed('All states completed.')